In [1]:
import requests
import json
import time
import os
from pathlib import Path
from os import mkdir
import youtube_dl

##### Завантажую аудіо зі списків відтворення у Ютуб

In [25]:
links = ['https://www.youtube.com/watch?v=eQ0hl7y1leU&list=PL1neMztLSbMMKbdNsHuwor7JQ02aKYVQn',
'https://www.youtube.com/watch?v=PEBaZSXVngQ&list=PL1neMztLSbMNYWEfQIyaAummnEVXMZLSI',
'https://www.youtube.com/watch?v=fmDqPVLPTWM&list=PL1neMztLSbMOnPCMvVW95dc7GIFE-xpvR',
'https://www.youtube.com/watch?v=GXIw1w3EYYU&list=PL1neMztLSbMPHZJg5NVR5NtvebNCOwG_F']


ydl_opts = { 'format': 'bestaudio[ext=m4a]',
            'match_filter': youtube_dl.utils.match_filter_func("duration < 180"),
            'outtmpl': 'data/raw/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s' }


with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    for link in links:
        ydl.download([link])    

##### Відправляю аудіо на сервер speech2text

In [2]:
def wav_to_text (path):
    print(path)
    auth = ("lakusta", "4d4RAQc6") 

    with open(path, 'rb') as f:
                
        post_data = {'submit': 'yes', 'lang': 'uk_ua'}

        response = requests.post("https://api.audionomic.com/v0.1/", 
                            auth = auth,
                            data = post_data, 
                            files = {'audiofile': f})
        print(response.text)

        url_w_punct = response.text.strip().replace('final.html', 'punct.html')
        text = requests.get(url_w_punct, auth = auth).text

        while text.startswith('Still recognizing'):
            time.sleep(10)
            text = requests.get(url_w_punct, auth=auth).text
                       
    return text

In [ ]:
#looping over files in directories recursively
for path in Path('data/wav/').rglob('*.wav'):

    text = wav_to_text(path)
    dir_name = path.parts[2]
    file_name = path.parts[3].replace('wav','txt')
    if not os.path.exists(f'data/text_punct/{dir_name}'):
        mkdir(f'data/text_punct/{dir_name}')
    with open(f'data/text_punct/{dir_name}/{file_name}', 'w+') as f:
        f.write(text)
        